In [3]:
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

internal_urls = set()
external_urls = set()

def is_valid(url):
    parsed = urlparse(url)
    
    if parsed.scheme.startswith('http') and bool(parsed.netloc):
        return True
    else:
        return False

def get_all_website_links(url):
    urls = set()
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    
    total_internal = 0
    total_external = 0
        
    for a_tag in soup.findAll("a"):
        
        href = a_tag.attrs.get("href")
        
        if href == "" or href is None:
            # href empty tag
            continue
        # join the URL if it's relative (not absolute link)
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        # remove URL GET parameters, URL fragments, etc.
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid(href):
            # not a valid URL
            continue
            
        if href in internal_urls:
            total_internal = total_internal + 1
            # already in the set
            continue
            
        if domain_name not in href:
            total_external = total_external + 1
            # external link
            if href not in external_urls:
#                 print(f"[!] External link: {href}")
                external_urls.add(href)
            continue
            
#         print(f"[*] Internal link: {href}")
        urls.add(href)
        internal_urls.add(href)
        total_internal = total_internal + 1
        
#     print("[+] Total Internal links:", len(internal_urls))
#     print("[+] Total External links:", len(external_urls))
    
    print("[+] Total Internal links:", total_internal)
    print("[+] Total External links:", total_external)
    
    return total_internal, total_external

In [4]:
df = pd.DataFrame({
    'Url':['https://www.bouwmaat.nl/bouwmaterialen','https://www.hornbach.nl/shop/Bouwmateriaal/S4471/artikeloverzicht.html','https://www.bouwbestel.nl/bouwmaterialen.html'],
    'Internal':[0,0,0],
    'External':[0,0,0]
})

In [5]:
df['Internal'] = 0
df['External'] = 0

counter = 0

for i in df.Url:
    gadget = get_all_website_links(i)
    df.Internal[counter] = gadget[0]
    df.External[counter] = gadget[1]
    counter = counter + 1

[+] Total Internal links: 1425
[+] Total External links: 11


<ipython-input-5-6f221121852b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Internal[counter] = gadget[0]
<ipython-input-5-6f221121852b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.External[counter] = gadget[1]


[+] Total Internal links: 217
[+] Total External links: 7
[+] Total Internal links: 2594
[+] Total External links: 14


In [6]:
df

,Url,Internal,External
0,https://www.bouwmaat.nl/bouwmaterialen,1425,11
1,https://www.hornbach.nl/shop/Bouwmateriaal/S44...,217,7
2,https://www.bouwbestel.nl/bouwmaterialen.html,2594,14
